In [0]:
BASE_PATH = "teste-1"
EXERCISE_NAME = "workshop-exercise-1"


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
import random


# Define the schema for the generated events
schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("merchant_id", StringType(), True),
    StructField("order_value", FloatType(), True),
    StructField("timestamp", TimestampType(), True),  # Add timestamp to the schema
])

# Define a generator function to create random events
def generate_merchant_id():
    # Update the choices for you favorite restaurants
    return random.choice(["A", "B", "C"])

def generate_order_value():
    return float(random.uniform(10, 100))

# Register the UDFs to be used with the streaming DataFrame
generate_merchant_id_udf = udf(generate_merchant_id, StringType())
generate_order_value_udf = udf(generate_order_value, FloatType())

spark.udf.register("generate_merchant_id", generate_merchant_id, StringType())
spark.udf.register("generate_order_value", generate_order_value, FloatType())

# Create a streaming DataFrame using the generator and schema
streaming_df = spark \
    .readStream \
    .format("rate") \
    .option("rowsPerSecond", 1) \
    .load() \
    .selectExpr("value as order_id")

# Apply the UDFs to generate the event data
streaming_df = streaming_df \
    .withColumn("merchant_id", generate_merchant_id_udf()) \
    .withColumn("order_value", generate_order_value_udf()) \
    .withColumn("timestamp", current_timestamp())  # Add current timestamp

# Write the streaming DataFrame to a Delta table
query = streaming_df \
    .writeStream \
    .outputMode("append") \
    .format("delta") \
    .option("checkpointLocation", f"/{BASE_PATH}/{EXERCISE_NAME}/checkpoint") \
    .option("path", f"{BASE_PATH}/{EXERCISE_NAME}/data") \
    .start()

# Wait for the stream to end
# query.awaitTermination()

In [0]:
df_events = spark.read.format("delta").load(f"/{BASE_PATH}/{EXERCISE_NAME}/data")

display(df_events)

order_id,merchant_id,order_value,timestamp
2,A,30.97563,2023-05-17T00:59:19.185+0000
10,A,78.17026,2023-05-17T00:59:19.185+0000
18,B,52.84361,2023-05-17T00:59:19.185+0000
26,C,92.21353,2023-05-17T00:59:19.185+0000
34,A,88.6409,2023-05-17T00:59:19.185+0000
42,B,25.87736,2023-05-17T00:59:19.185+0000
50,A,37.87125,2023-05-17T00:59:19.185+0000
1,B,46.569294,2023-05-17T00:59:19.185+0000
9,B,57.50949,2023-05-17T00:59:19.185+0000
17,C,82.30402,2023-05-17T00:59:19.185+0000


### Exercise 1: Read Table Using Streaming and use group by function
In this exercise, you will learn how to read a streaming Delta table in PySpark and perform a windowing aggregation. Specifically, you will count the number of orders for each merchant and also get the average ticket for each merchant in the dataset.

Here is your task: Count the number of orders for each merchant and the average ticket, using the "merchant_id" field as the key. Use the groupBy function in PySpark to do this.

In [0]:
# Your code goes here 

### Exercise 2: Windowing with Watermark
In the second exercise, you will be using a technique known as "windowing with watermark".

Your task: Read the same Delta table, group the data by "merchant_id" and count the number of orders in each window of time, say 1 minute and with a step of 10 seconds. Use a watermark of 10 seconds to allow late data.

In [0]:
# Your code goes here 

Exercise 3: Using Arbitrary Statefull Aggregation

In this exercise, you will be using arbitrary stateful processing. This allows you to perform more complex calculations over the data stream.

Your task: Implement the same aggregation as in the previous exercises, but this time using arbitrary stateful processing.


> Remember, the flatMapGroupWithState only works in `Scala`.

In [0]:
%scala

// your code goes here
